In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount = True)
import os
os.chdir('/content/drive/My Drive')

In [ ]:
os.listdir()

['Colab Notebooks',
 'Help.zip',
 'Negative_Data.zip',
 'help_trimmed_train_x.pickle',
 'help_trimmed_test_y.pickle',
 'help_trimmed_train_y.pickle',
 'help_trimmed_test_x.pickle',
 'help_trimmed.h5',
 'Help',
 'Negative',
 'new.wav',
 'model1.json',
 'model1.h5',
 'model2.json',
 'model2.h5',
 'model3.json',
 'model3.h5',
 'Models',
 'speech_commands_v0.02.tar.gz',
 'Google_Speech',
 'Google_Noise',
 'model3_1.json',
 'model3_1.h5',
 'Google_Sample',
 'google_trimmed_train_x.pickle',
 'google_trimmed_train_y.pickle',
 'google_mix_help_train_y.pickle',
 'train_x_extra_noise.pickle',
 'google_mix_help_train_x.pickle',
 'model_a.json',
 'model_a.h5',
 'Hey Siri Models.gslides',
 'Wake-word Detection Update.gslides',
 'Object Detection.gslides',
 'Images',
 'coco.names',
 'yolov3.weights',
 'futur.ttf',
 'Google_Sample2',
 'Vocal_Noise']

In [ ]:
# !tar -xf speech_commands_v0.02.tar.gz -C ./Google_Speech

In [ ]:
import numpy as np
import os
import matplotlib.pyplot as plt
from scipy.io.wavfile import read
from scipy.io.wavfile import write

import random
from random import randint

import wave

labels = ["yes", "no", "up", "down", "left", "right", "on", "off", "stop", "go", "zero", "one", "two", "three", "four",
"five", "six", "seven", "eight", "nine", "bed", "bird", "cat", "dog", "happy", "house", "marvin", "sheila", "tree", "wow"]

In [ ]:
#Cutting a random section
def cut_random_section(noise2, size2):
    size21 = noise2.size
    starting_point2 = randint(0,(noise2.size - size2))
    end_point2 = starting_point2 + size2
    noise_cut_part2 = noise2[starting_point2:end_point2]
    return noise_cut_part2

#Mixing
def mix(audio1, noise1, snr1):
    audio_max = max(audio1)
    if audio_max==0:
        audio_max = int(np.random.uniform(0.7,1)*32767)
    audio1 = audio1*1.
    audio1 = audio1/audio_max
    print(audio1,noise1)
    noise1 = cut_random_section(noise1, audio1.size)
    noise1 = noise1*1.
    print(noise1)
    print(audio1.size, noise1.size)
    noise1 = noise1/max(noise1)
    gain = pow(10,(snr1/10.))
    numerator = np.mean(abs(audio1)**2)
    denominator = numerator/gain
    noise_power = np.mean(abs(noise1)**2)
    mult_value = (denominator/noise_power)**0.5
    noisy1 = audio1 + noise1*mult_value
    if max(audio1)==0:
        noisy1 = noise1
    else:    
        noisy1 = noisy1/max(noisy1)
    noisy1 = np.array(noisy1*audio_max, dtype='int16')
    # print(audio_max)
    return noise1*mult_value, mult_value, noisy1

def readwav(file):
    """
    Read a wav file.
    Returns the frame rate, sample width (in bytes) and a numpy array
    containing the data.
    This function does not read compressed wav files.
    """
    wav = wave.open(file)
    rate = wav.getframerate()
    nchannels = wav.getnchannels()
    sampwidth = wav.getsampwidth()
    nframes = wav.getnframes()
    data = wav.readframes(nframes)
    wav.close()
    array = _wav2array(nchannels, sampwidth, data)
    return rate, sampwidth, array

In [ ]:
# !pip install soundfile
# import soundfile

# def convertAllFilesInDirectoryTo16Bit(directory):
#     for file in os.listdir(directory):
#          if(file.endswith('.wav')):
#              nameSolo = file.rsplit('.', 1)[0]
#              print(directory + nameSolo )
#              data, samplerate = soundfile.read(directory + file)                

#              soundfile.write(directory + file, data, samplerate, subtype='PCM_16')
#              print("converting " + file + "to 16 - bit")

# convertAllFilesInDirectoryTo16Bit('Vocal_Noise/')

In [ ]:
snr_dB = 10

noise_files = os.listdir('Vocal_Noise')
for label in labels:
  try:  
    os.mkdir("./Google_Sample2/" + str(label))
  except:
    0+0
  files = os.listdir("Google_Speech/" + str(label))

  # #Sample and store
  # if len(files) > 30:
  #   sampling = random.choices(files, k=30)
  # else:
  #   sampling = files

  # for temp in sampling:
  #   fs,x = read('Google_Speech/' + label + '/' + temp)
  #   os.chdir("./Google_Sample/" + str(label))
  #   write(temp[:-4] + '_' + label, fs, x)
  #   os.chdir("../../")  
  # print(label, len(sampling))

  #Sample, add noise and store
  for noise in noise_files:
    fs, noise_file = read('Vocal_Noise/' + noise)

    if len(files) > 30:
      sampling = random.choices(files, k=30)
    else:
      sampling = files

    for temp in sampling:
      fs,x = read('Google_Speech/' + label + '/' + temp)
      noise_temp, mult_value, noisy = mix(x, noise_file, snr_dB)
      os.chdir("./Google_Sample2/" + str(label))
      write(temp[:-4] + '_' + label + '_' + noise[:-4], fs, noisy) 
      os.chdir("../../")   
    print(noise)

[-7.93210121e-05 -4.75926073e-04 -3.17284049e-04 ... -7.13889109e-04
 -3.96605061e-04 -3.96605061e-04] [[   0   -1]
 [  -1    0]
 [  -1    0]
 ...
 [-568 -341]
 [-599 -314]
 [-491 -353]]
[]
16000 0


ValueError: ignored

In [ ]:
import librosa
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense,LSTM,GlobalMaxPool1D, Bidirectional 
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import TensorBoard
import numpy as np
import os
from sklearn.utils import shuffle
from sklearn import metrics
import pickle
from scipy.io.wavfile import read,write
import pickle

from datetime import datetime
from packaging import version

tf.keras.backend.set_floatx('float64')

print("TensorFlow version: ", tf.__version__)

TensorFlow version:  2.2.0


In [ ]:
INPUT_SHAPE = (126, 40)
dropout = 0.2
var = 0
folders = [("Google_Sample/" + str(label), False) for label in labels]
print(folders)

[('Google_Sample/yes', False), ('Google_Sample/no', False), ('Google_Sample/up', False), ('Google_Sample/down', False), ('Google_Sample/left', False), ('Google_Sample/right', False), ('Google_Sample/on', False), ('Google_Sample/off', False), ('Google_Sample/stop', False), ('Google_Sample/go', False), ('Google_Sample/zero', False), ('Google_Sample/one', False), ('Google_Sample/two', False), ('Google_Sample/three', False), ('Google_Sample/four', False), ('Google_Sample/five', False), ('Google_Sample/six', False), ('Google_Sample/seven', False), ('Google_Sample/eight', False), ('Google_Sample/nine', False), ('Google_Sample/bed', False), ('Google_Sample/bird', False), ('Google_Sample/cat', False), ('Google_Sample/dog', False), ('Google_Sample/happy', False), ('Google_Sample/house', False), ('Google_Sample/marvin', False), ('Google_Sample/sheila', False), ('Google_Sample/tree', False), ('Google_Sample/wow', False)]


In [ ]:
def count_files(folder, extension):
	count = 0
	for file in os.listdir(folder):
		if file.endswith(extension):
			file_path = os.path.join(folder, file)
			count += 1
	return count

def load_data_folder(folder, is_keyword):
  num_samples = len(os.listdir(folder))
  data_X = np.zeros((num_samples, INPUT_SHAPE[0], INPUT_SHAPE[1]), dtype=np.float64)
  data_Y = np.zeros((num_samples), dtype=np.float64)
  count = 0
  for file in os.listdir(folder):
    # if file.endswith('.wav'):
      file_path = os.path.join(folder, file)
      y, sr = librosa.load(file_path,sr=None)
      temp = librosa.feature.mfcc(y=y, sr=sr, hop_length=128, n_fft=256, n_mfcc=20)
      if temp.shape != (20, 126):
        continue
      idx = (INPUT_SHAPE[0]-temp.shape[1])//2
      mfcc = np.zeros((20,126))
      if temp.shape[1]%2 == 0:
        mfcc[:, idx: INPUT_SHAPE[0] - idx] = temp
      else:
        mfcc[:, idx: INPUT_SHAPE[0] - idx - 1] = temp
      mfcc_delta = librosa.feature.delta(mfcc)[:10, :]
      mfcc_double_delta = librosa.feature.delta(mfcc, order=2)[:10, :]
      data_X[count, :, :20] = mfcc.T
      data_X[count, :, 20:30] = mfcc_delta.T
      data_X[count, :, 30:] = mfcc_double_delta.T
      data_Y[count] = int(is_keyword)
      count += 1
      if count%50==0:
        print(count)
  return data_X, data_Y

def load_data(folders):
	num_samples = sum(len(os.listdir(folder)) for (folder,_) in folders)
	data_X = np.zeros((num_samples, INPUT_SHAPE[0], INPUT_SHAPE[1]), dtype=np.float64)
	data_Y = np.zeros((num_samples), dtype=np.float64)
	count = 0
	for folder, is_keyword in folders:
		num_samples_folder = len(os.listdir(folder))
		data_X[count:count+num_samples_folder, :, :], data_Y[count:count+num_samples_folder] = (load_data_folder(folder, is_keyword))
		count += num_samples_folder
	return shuffle(data_X, data_Y, random_state=0)

train_X0, train_Y0 = load_data(folders)
print("Train data extracted")
print(train_X0.shape, train_Y0.shape)

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
import pickle

with open("google_vocal_train_x.pickle", "wb") as f:
  pickle.dump(train_X0, f)
print("Train set features done")
with open("google_vocal_train_y.pickle", "wb") as f:
  pickle.dump(train_Y0, f)
print("Train set labels done")

Train set features done
Train set labels done
